In [38]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import torch
import torch.nn as nn
import time




In [52]:
# get data and normilize it

file_csv = pd.read_csv("../csv_data/AAPL.csv")
# file_csv
# file_csv.dropna(subset = ["polarity"], inplace=True)
# file_csv.dropna(subset = ["subjectivity"], inplace=True)
# file_csv.dropna(subset = ["prices"], inplace=True)

# polarity = file_csv["polarity"].values
# subjectivity = file_csv["subjectivity"].values
prices = file_csv[["prices"]]


scaler = MinMaxScaler(feature_range=(-1, 1))
file_csv["prices"] = scaler.fit_transform(file_csv["prices"].values.reshape(-1,1))


In [53]:
# split data
def split_data(stock, lookback):
    data_raw = stock.to_numpy() # convert to numpy array
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]
lookback = 20 # choose sequence length
x_train, y_train, x_test, y_test = split_data(prices, lookback)

In [54]:
# transform sets to tensors
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
y_train_gru = torch.from_numpy(y_train).type(torch.Tensor)
y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)

In [65]:
# common values
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 500

In [66]:
# Define the model 
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [71]:
# create model instance, optimizer, and criterion
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.1)

In [72]:

hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []
for t in range(num_epochs):
    y_train_pred = model(x_train)
    loss = criterion(y_train_pred, y_train_lstm)
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    
training_time = time.time()-start_time
print("Training time: {}".format(training_time))

Epoch  0 MSE:  4307.7265625
Epoch  1 MSE:  3907.979248046875
Epoch  2 MSE:  3696.928955078125
Epoch  3 MSE:  3493.86962890625
Epoch  4 MSE:  3286.265869140625
Epoch  5 MSE:  3086.62890625
Epoch  6 MSE:  2892.189697265625
Epoch  7 MSE:  2703.77685546875
Epoch  8 MSE:  2522.28759765625
Epoch  9 MSE:  2348.337158203125
Epoch  10 MSE:  2182.344482421875
Epoch  11 MSE:  2024.5931396484375
Epoch  12 MSE:  1875.263916015625
Epoch  13 MSE:  1734.459228515625
Epoch  14 MSE:  1602.216796875
Epoch  15 MSE:  1478.517333984375
Epoch  16 MSE:  1363.2933349609375
Epoch  17 MSE:  1256.432373046875
Epoch  18 MSE:  1157.78076171875
Epoch  19 MSE:  1067.1483154296875
Epoch  20 MSE:  984.3091430664062
Epoch  21 MSE:  909.0059204101562
Epoch  22 MSE:  840.9525146484375
Epoch  23 MSE:  779.8357543945312
Epoch  24 MSE:  725.3204345703125
Epoch  25 MSE:  677.05029296875
Epoch  26 MSE:  634.6526489257812
Epoch  27 MSE:  597.7422485351562
Epoch  28 MSE:  565.9236450195312
Epoch  29 MSE:  538.7962036132812
Epoch